In [ ]:
# main.py

import uuid
import math
import random
from datetime import datetime, timedelta
from dataclasses import dataclass, field

# Importamos o nosso módulo de análise de risco
import risk_analysis_module

# --- 1. MODELOS DE DADOS DA APLICAÇÃO ---

# ... (todas as classes anteriores: Wallet, Transaction, Company, etc. permanecem aqui)
# Adicionamos a nova classe para o Mercado Secundário

@dataclass
class SecondaryMarketListing:
    listing_id: uuid.UUID = field(default_factory=uuid.uuid4)
    investment_id: uuid.UUID = None # O investimento que está sendo vendido
    seller_id: uuid.UUID = None # O ID do investidor vendedor
    qtd_quota_for_sale: int = 0
    price_per_quota: float = 0.0 # O vendedor define o preço
    status: str = "OPEN" # OPEN, SOLD, CANCELLED
    created_at: datetime = field(default_factory=datetime.now)

# ... (outras dataclasses inalteradas)
@dataclass
class Wallet:
    wallet_id: uuid.UUID = field(default_factory=uuid.uuid4)
    owner_id: uuid.UUID = None
    balance: float = 0.0
    currency: str = "BRL"

@dataclass
class Company:
    company_id: uuid.UUID = field(default_factory=uuid.uuid4)
    company_name: str = "TechCorp S.A."
    annual_revenue: int = 1_200_000
    founded_date: datetime = datetime(2018, 5, 20)
    financial_data: dict = field(default_factory=lambda: {
        "cpf_score_socio": 850, "divida_total": 300_000, "saldo_medio_diario": 100_000,
        "faturamento_medio_mensal": 100_000, "estresse_caixa_dias": 2, "valor_maior_cliente": 60_000,
        "faturamento_total_periodo": 600_000, "cnpj_score_api": 900
    })

@dataclass
class CreditRequest:
    request_id: uuid.UUID = field(default_factory=uuid.uuid4)
    company_id: uuid.UUID = None
    amount: int = 250_000
    period: int = 12
    status: str = "PENDING"
    created_at: datetime = field(default_factory=datetime.now)

@dataclass
class Opportunity:
    opportunity_id: uuid.UUID = field(default_factory=uuid.uuid4)
    request_id: uuid.UUID = None
    total_quota: int = 0
    accumulated_quota: int = 0
    fixed_quota: int = 200
    interest_rate: float = 0.0
    status: str = "OPEN"
    created_at: datetime = field(default_factory=datetime.now)

@dataclass
class Investment:
    investment_id: uuid.UUID = field(default_factory=uuid.uuid4)
    investor_id: uuid.UUID = None
    opportunity_id: uuid.UUID = None
    qtd_quota: int = 0
    created_at: datetime = field(default_factory=datetime.now)

@dataclass
class Repayment:
    repayment_id: uuid.UUID = field(default_factory=uuid.uuid4)
    opportunity_id: uuid.UUID = None
    amount: float = 0.0
    due_date: datetime = None
    status: str = "PENDING"

@dataclass
class Distribution:
    distribution_id: uuid.UUID = field(default_factory=uuid.uuid4)
    investment_id: uuid.UUID = None
    repayment_id: uuid.UUID = None
    principal_amount: float = 0.0
    interest_amount: float = 0.0
    status: str = "PENDING"
    
# --- 2. CONSTANTES DE NEGÓCIO ---

SECONDARY_MARKET_FEE_PERCENTAGE = 0.02 # Taxa de 2% sobre a venda

# --- 3. FUNÇÕES DE FLUXO DE NEGÓCIO ---

# ... (todas as funções anteriores: process_credit_request, execute_payout, etc. permanecem aqui)

def process_credit_request(request: CreditRequest, company: Company) -> Opportunity | None:
    # ... (função inalterada)
    print("--- Iniciando Análise de Risco ---")
    idade_empresa_anos = (datetime.now() - company.founded_date).days / 365.25
    score, classificacao = risk_analysis_module.calculate_risk_score(
        idade_empresa=idade_empresa_anos,
        cpf_score=company.financial_data["cpf_score_socio"],
        divida_total=company.financial_data["divida_total"],
        faturamento_anual=company.annual_revenue,
        saldo_medio_diario=company.financial_data["saldo_medio_diario"],
        faturamento_medio_mensal=company.financial_data["faturamento_medio_mensal"],
        estresse_caixa_dias=company.financial_data["estresse_caixa_dias"],
        valor_maior_cliente=company.financial_data["valor_maior_cliente"],
        faturamento_total_periodo=company.financial_data["faturamento_total_periodo"],
        cnpj_score_api=company.financial_data["cnpj_score_api"]
    )
    print(f"Análise concluída. Score: {score}, Classificação: {classificacao}")

    if classificacao == 'automatically_reproved':
        print("\n--- Resultado: Solicitação de Crédito REPROVADA ---")
        request.status = "REPROVED"
        return None
    
    print("\n--- Resultado: Solicitação de Crédito APROVADA ---")
    request.status = "APPROVED"
    print("--- Criando Oportunidade de Investimento ---")
    taxa_juros = risk_analysis_module.calcular_taxa_juros_anual(
        risco_final=score, prazo_meses=request.period, valor_solicitado=request.amount
    )
    qtd_cotas = math.ceil(request.amount / 200)
    nova_oportunidade = Opportunity(
        request_id=request.request_id,
        total_quota=qtd_cotas,
        interest_rate=round(taxa_juros, 4)
    )
    print("Oportunidade criada com sucesso!")
    return nova_oportunidade


def execute_payout(request: CreditRequest, opportunity: Opportunity, company_wallet: Wallet, admin_wallet: Wallet):
    # ... (função inalterada)
    print("\n--- Executando Payout para a Empresa ---")
    PLATFORM_FEE_PERCENTAGE = 0.05
    requested_amount = request.amount
    platform_fee = requested_amount * PLATFORM_FEE_PERCENTAGE
    net_payout_to_company = requested_amount - platform_fee
    
    admin_wallet.balance += platform_fee
    company_wallet.balance += net_payout_to_company
    
    opportunity.status = "FUNDED"
    request.status = "COMPLETED"
    print(f"Pago R$ {net_payout_to_company:,.2f} para a empresa. Taxa de R$ {platform_fee:,.2f} para a plataforma.")


def simulate_investment_capture(opportunity: Opportunity, request: CreditRequest, company_wallet: Wallet, admin_wallet: Wallet, investor_wallets: list[Wallet]) -> list[Investment]:
    # ... (função inalterada)
    print("\n--- Iniciando Captação de Investimentos ---")
    investments = []
    
    while opportunity.accumulated_quota < opportunity.total_quota:
        investor_wallet = random.choice(investor_wallets)
        max_quotas_to_buy = opportunity.total_quota - opportunity.accumulated_quota
        quotas_to_buy = random.randint(1, max(1, max_quotas_to_buy // 2))

        if opportunity.accumulated_quota + quotas_to_buy > opportunity.total_quota:
            quotas_to_buy = opportunity.total_quota - opportunity.accumulated_quota
        
        investment_value = quotas_to_buy * opportunity.fixed_quota
        if investor_wallet.balance >= investment_value:
            investor_wallet.balance -= investment_value
            opportunity.accumulated_quota += quotas_to_buy
            
            new_investment = Investment(
                investor_id=investor_wallet.owner_id,
                opportunity_id=opportunity.opportunity_id,
                qtd_quota=quotas_to_buy
            )
            investments.append(new_investment)
            # print(f"Investidor {investor_wallet.owner_id.hex[:6]}... comprou {quotas_to_buy} cota(s).")
    
    print("\n--- META DE CAPTAÇÃO ATINGIDA! ---")
    execute_payout(request, opportunity, company_wallet, admin_wallet)
    return investments

def create_repayment_schedule(request: CreditRequest, opportunity: Opportunity) -> list[Repayment]:
    # ... (função inalterada)
    return [] # Simplificando para focar no mercado secundário

# --- NOVAS FUNÇÕES PARA O MERCADO SECUNDÁRIO ---

def create_secondary_market_listing(
    investment_to_sell: Investment,
    price_per_quota: float,
    listings: list[SecondaryMarketListing]
) -> SecondaryMarketListing:
    """
    Cria uma oferta de venda de cotas no mercado secundário.
    """
    print("\n--- Criando Oferta no Mercado Secundário ---")
    
    # Validação (Em um sistema real, haveria mais checagens)
    if investment_to_sell.qtd_quota <= 0:
        print("Erro: Investimento sem cotas para vender.")
        return None
        
    listing = SecondaryMarketListing(
        investment_id=investment_to_sell.investment_id,
        seller_id=investment_to_sell.investor_id,
        qtd_quota_for_sale=investment_to_sell.qtd_quota,
        price_per_quota=price_per_quota
    )
    listings.append(listing)
    
    print(f"Investidor {listing.seller_id.hex[:6]}... colocou {listing.qtd_quota_for_sale} cotas à venda por R$ {listing.price_per_quota:,.2f} cada.")
    return listing

def execute_secondary_market_trade(
    listing: SecondaryMarketListing,
    buyer_wallet: Wallet,
    seller_wallet: Wallet,
    admin_wallet: Wallet,
    investments: list[Investment]
):
    """
    Executa a troca de cotas entre dois investidores.
    """
    print("\n--- Executando Troca no Mercado Secundário ---")
    
    # 1. Cálculos
    total_price = listing.qtd_quota_for_sale * listing.price_per_quota
    platform_fee = total_price * SECONDARY_MARKET_FEE_PERCENTAGE
    net_to_seller = total_price - platform_fee
    
    print(f"Comprador {buyer_wallet.owner_id.hex[:6]}... comprando {listing.qtd_quota_for_sale} cotas.")
    print(f"Valor total da operação: R$ {total_price:,.2f}")
    
    # 2. Validação de saldo
    if buyer_wallet.balance < total_price:
        print(f"Falha na transação: Saldo insuficiente do comprador (Saldo: R$ {buyer_wallet.balance:,.2f})")
        return

    # 3. Transações financeiras
    buyer_wallet.balance -= total_price
    admin_wallet.balance += platform_fee
    seller_wallet.balance += net_to_seller
    
    print(f"Taxa da plataforma (2%): R$ {platform_fee:,.2f} -> Saldo Admin: R$ {admin_wallet.balance:,.2f}")
    print(f"Valor líquido para vendedor: R$ {net_to_seller:,.2f} -> Saldo Vendedor: R$ {seller_wallet.balance:,.2f}")
    
    # 4. Transferência de propriedade das cotas
    # Encontra o investimento original do vendedor
    original_investment = next((inv for inv in investments if inv.investment_id == listing.investment_id), None)
    
    if original_investment:
        # Cria um novo registro de investimento para o comprador
        new_investment_for_buyer = Investment(
            investor_id=buyer_wallet.owner_id,
            opportunity_id=original_investment.opportunity_id,
            qtd_quota=listing.qtd_quota_for_sale
        )
        investments.append(new_investment_for_buyer)
        
        # Remove as cotas do vendedor
        original_investment.qtd_quota -= listing.qtd_quota_for_sale
    
    # 5. Atualiza o status da oferta
    listing.status = "SOLD"
    print(f"Propriedade de {listing.qtd_quota_for_sale} cotas transferida para o comprador.")
    print("--- Troca Concluída com Sucesso ---")

# --- EXECUÇÃO DA SIMULAÇÃO COMPLETA ---
if __name__ == "__main__":
    # 1. Configuração inicial
    empresa_solicitante = Company()
    solicitacao_de_credito = CreditRequest(company_id=empresa_solicitante.company_id)
    
    carteira_da_empresa = Wallet(owner_id=empresa_solicitante.company_id, balance=150000.0)
    carteira_administrativa = Wallet(owner_id=uuid.uuid4(), balance=0.0)
    # Criamos carteiras para 5 investidores com saldos iniciais
    carteiras_dos_investidores = [Wallet(owner_id=uuid.uuid4(), balance=100000.0) for _ in range(5)]

    print("--- INÍCIO DA SIMULAÇÃO QINVEST ---")
    
    # 2. Processa o crédito e cria a oportunidade
    oportunidade_criada = process_credit_request(solicitacao_de_credito, empresa_solicitante)
    
    if oportunidade_criada:
        # 3. Simula a captação dos investimentos (mercado primário)
        investimentos_realizados = simulate_investment_capture(
            oportunidade_criada, solicitacao_de_credito, carteira_da_empresa, carteira_administrativa, carteiras_dos_investidores
        )
        
        print("\n--- ESTADO APÓS MERCADO PRIMÁRIO ---")
        for i, inv in enumerate(investimentos_realizados):
            print(f"  -> Investimento {i+1}: {inv.qtd_quota} cotas, Dono: {inv.investor_id.hex[:6]}...")
        
        # 4. SIMULAÇÃO DO MERCADO SECUNDÁRIO
        mercado_secundario_listings = []
        
        # O primeiro investidor decide vender todo o seu investimento
        if investimentos_realizados:
            investimento_a_venda = investimentos_realizados[0]
            vendedor_wallet = next(w for w in carteiras_dos_investidores if w.owner_id == investimento_a_venda.investor_id)
            
            # O vendedor decide vender com um pequeno lucro (ágio)
            preco_de_venda_por_cota = 205.50 
            
            oferta = create_secondary_market_listing(
                investimento_a_venda, preco_de_venda_por_cota, mercado_secundario_listings
            )
            
            if oferta:
                # O segundo investidor decide comprar
                comprador_wallet = carteiras_dos_investidores[1]
                
                print("\nSaldos ANTES da troca:")
                print(f"  -> Saldo Vendedor: R$ {vendedor_wallet.balance:,.2f}")
                print(f"  -> Saldo Comprador: R$ {comprador_wallet.balance:,.2f}")
                print(f"  -> Saldo Admin: R$ {carteira_administrativa.balance:,.2f}")
                
                execute_secondary_market_trade(
                    oferta, comprador_wallet, vendedor_wallet, carteira_administrativa, investimentos_realizados
                )

                print("\n--- ESTADO FINAL ---")
                print("Saldos APÓS a troca:")
                print(f"  -> Saldo Vendedor: R$ {vendedor_wallet.balance:,.2f}")
                print(f"  -> Saldo Comprador: R$ {comprador_wallet.balance:,.2f}")
                print(f"  -> Saldo Admin: R$ {carteira_administrativa.balance:,.2f}")
                
                print("\nPropriedade final das cotas:")
                for i, inv in enumerate(investimentos_realizados):
                    if inv.qtd_quota > 0: # Mostra apenas investimentos com cotas
                        print(f"  -> Investimento {i+1}: {inv.qtd_quota} cotas, Dono: {inv.investor_id.hex[:6]}...")